In [ ]:
import pandas as pd
import numpy as np
from utils.stofn import *
from utils.eldi import *
from utils.dreifing import *
from utils.gogn import *
import random

In [ ]:
data = {}
data['rivers'] = getRivers()
data['eldi'] =  getFarms()
data['distances'] = getDistances()

In [ ]:
ITERS = 1000

EVENTS_PER_YEAR = 1.75 # Average number of escape events per year
SIZE_PROPORTION = 0.67 # Proportion of Early vs Late escapees
ESCAPES_PER_TON = 0.5 # Amount of escapees per 1000 ton

LATE_RETURNS_PROP = 0.0016 # Proportion of Late escapees that return to rivers (0.16%)
EARLY_RETURNS_PROP = 0.0007 # Proportion of Early escapees that return to rivers (0.07%)
EARLY_YEARLY_DISTR = [0, 30/56, 17/56, 9/56] # Early returns distributed over four years

LATE_PROPORTION = 0.2
LATE_LENGTH = 240

EARLY_PROPORTION = 0.5
EARLY_LENGTH = 140

In [ ]:
def runSimulation(ITERS = ITERS, \
            LATE_PROPORTION = LATE_PROPORTION,\
            EARLY_PROPORTION = EARLY_PROPORTION,\
            LATE_LENGTH = LATE_LENGTH, \
            EARLY_LENGTH = EARLY_LENGTH, \
            EVENTS_PER_YEAR = EVENTS_PER_YEAR, \
            ESCAPES_PER_TON = ESCAPES_PER_TON, \
            SIZE_PROPORTION = SIZE_PROPORTION, \
            LATE_RETURNS_PROP = LATE_RETURNS_PROP, \
            EARLY_RETURNS_PROP = EARLY_RETURNS_PROP, \
            EARLY_YEARLY_DISTR = EARLY_YEARLY_DISTR):
        stofnar = stofnstaerdir(data,ITERS)
        escSchedule = calcEscapeEvents(data,ITERS, EVENTS_PER_YEAR)
        farmEvents = splitEvents(data, escSchedule, ITERS)
        farmEventsEarly, farmEventsLate = splitFarmEvents(data, farmEvents,ITERS,SIZE_PROPORTION)
        farmNumbersEarly, farmNumbersLate = getSizeOfEvents(data,farmEventsEarly, farmEventsLate, ESCAPES_PER_TON, EVENTS_PER_YEAR)
        farmEarlyReturns, farmLateReturns = getNumberOfReturners(data, farmNumbersEarly, farmNumbersLate, ITERS, LATE_RETURNS_PROP, EARLY_RETURNS_PROP, EARLY_YEARLY_DISTR)
        results = getResults(data, stofnar, farmEarlyReturns, farmLateReturns, ITERS,LATE_PROPORTION,EARLY_PROPORTION,LATE_LENGTH,EARLY_LENGTH )
        return results
    

In [ ]:
avg = pd.DataFrame([])
count4 = pd.DataFrame([])
count4over3 = pd.DataFrame([])

start = 180 # fyrsta gildi til að prófa
stop =  280 # síðasta gildi til að prófa
num = 10    # fjöldi gilda til að prófa
values = np.linspace(start,stop,num)
for val in values:
    print(val)
    results = runSimulation(LATE_LENGTH = val) ## Parameter to change
    avg[val] = results.mean(axis=0)
    count4[val] = ((results>4)*1).mean(axis=0)*100
    count4over3 = ((results>4).rolling(3).mean()>4).mean(axis=0)*100



In [ ]:
avg.to_csv('data/sensAnalysis/Late_lengthavg.csv')  

In [ ]:
count4.to_csv('data/sensAnalysis/Late_lengtho4.csv') 

In [ ]:
count4over3.to_csv('data/sensAnalysis/Late_length4o3.csv') 